In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
datadir = '/home/mei/nas/docker/dataset/EICU/eicu-collaborative-research-database-2.0/'


In [5]:
def round_up(x, base=5):
    return base * round(x/base)

In [1]:
# # Import libraries
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import psycopg2
# import getpass
# import pdvega
# import seaborn as sns
# # for configuring connection 
# from configobj import ConfigObj
# import os

# %matplotlib inline

In [8]:
df = pd.read_csv(datadir + 'patient.csv.gz', compression='gzip')

In [9]:
df.head()

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
0,141168,128919,Female,70,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",152.4,15:54:00,...,Direct Admit,1,admit,84.3,85.8,03:50:00,3596,Death,Expired,002-34851
1,141178,128927,Female,52,Caucasian,60,83,NaN,162.6,08:56:00,...,Emergency Department,1,admit,54.4,54.4,09:18:00,8,Step-Down Unit (SDU),Alive,002-33870
2,141179,128927,Female,52,Caucasian,60,83,NaN,162.6,08:56:00,...,ICU to SDU,2,stepdown/other,NaN,60.4,19:20:00,2042,Home,Alive,002-33870
3,141194,128941,Male,68,Caucasian,73,92,"Sepsis, renal/UTI (including bladder)",180.3,18:18:40,...,Floor,1,admit,73.9,76.7,15:31:00,4813,Floor,Alive,002-5276
4,141196,128943,Male,71,Caucasian,67,109,NaN,162.6,20:21:00,...,ICU to SDU,2,stepdown/other,NaN,63.2,22:23:00,1463,Floor,Alive,002-37665


In [15]:
#selected patients that comes from 'direct and emergency dep.'
filtered_df = df[df['unitadmitsource'].isin(['Direct Admit', 'Emergency Department'])]

# Set indices
df_selected = df.set_index(['patientunitstayid'])
df_selected .sort_index(level = ['patientunitstayid'], inplace = True)
df_selected  = df_selected [['hospitaladmitoffset','gender','age','apacheadmissiondx','unitadmitsource','admissionweight','dischargeweight','unitdischargelocation',	'unitdischargestatus']]
# Resample every 5 mins
# df_selcted .rename(round_up, level = 'drugoffset', inplace = True)


In [16]:
df_selected .head()

,hospitaladmitoffset,gender,age,apacheadmissiondx,unitadmitsource,admissionweight,dischargeweight,unitdischargelocation,unitdischargestatus
patientunitstayid,,,,,,,,,
141168,0,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired
141178,-14,Female,52,NaN,Emergency Department,54.4,54.4,Step-Down Unit (SDU),Alive
141179,-22,Female,52,NaN,ICU to SDU,NaN,60.4,Home,Alive
141194,-780,Male,68,"Sepsis, renal/UTI (including bladder)",Floor,73.9,76.7,Floor,Alive
141196,-99,Male,71,NaN,ICU to SDU,NaN,63.2,Floor,Alive


In [18]:
# df_selected.to_csv('/home/mei/nas/docker/processedData/selected_patients.csv', index=False)
# print("Data exported successfully to 'selected_patients.csv'.")

Data exported successfully to 'selected_patients.csv'.


In [21]:
df_selected .head()

,hospitaladmitoffset,gender,age,apacheadmissiondx,unitadmitsource,admissionweight,dischargeweight,unitdischargelocation,unitdischargestatus
patientunitstayid,,,,,,,,,
141168,0,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired
141178,-14,Female,52,NaN,Emergency Department,54.4,54.4,Step-Down Unit (SDU),Alive
141179,-22,Female,52,NaN,ICU to SDU,NaN,60.4,Home,Alive
141194,-780,Male,68,"Sepsis, renal/UTI (including bladder)",Floor,73.9,76.7,Floor,Alive
141196,-99,Male,71,NaN,ICU to SDU,NaN,63.2,Floor,Alive


In [24]:
n_selected_patients=len(list(df_selected.index.get_level_values('patientunitstayid').unique()))
print("There are {} unique patientunitstayid in the selected patients with the selected unitadmitsource: emergency and direct.".format(n_selected_patients))

There are 200859 unique patientunitstayid in the selected patients with the selected unitadmitsource: emergency and direct.


In [2]:
# # Create a database connection using settings from config file
# config='../db/config.ini'

# # connection info
# conn_info = dict()
# if os.path.isfile(config):
#     config = ConfigObj(config)
#     conn_info["sqluser"] = config['username']
#     conn_info["sqlpass"] = config['password']
#     conn_info["sqlhost"] = config['host']
#     conn_info["sqlport"] = config['port']
#     conn_info["dbname"] = config['dbname']
#     conn_info["schema_name"] = config['schema_name']
# else:
#     conn_info["sqluser"] = 'postgres'
#     conn_info["sqlpass"] = ''
#     conn_info["sqlhost"] = 'localhost'
#     conn_info["sqlport"] = 5432
#     conn_info["dbname"] = 'eicu'
#     conn_info["schema_name"] = 'public,eicu_crd'
    
# # Connect to the eICU database
# print('Database: {}'.format(conn_info['dbname']))
# print('Username: {}'.format(conn_info["sqluser"]))
# if conn_info["sqlpass"] == '':
#     # try connecting without password, i.e. peer or OS authentication
#     try:
#         if (conn_info["sqlhost"] == 'localhost') & (conn_info["sqlport"]=='5432'):
#             con = psycopg2.connect(dbname=conn_info["dbname"],
#                                    user=conn_info["sqluser"])            
#         else:
#             con = psycopg2.connect(dbname=conn_info["dbname"],
#                                    host=conn_info["sqlhost"],
#                                    port=conn_info["sqlport"],
#                                    user=conn_info["sqluser"])
#     except:
#         conn_info["sqlpass"] = getpass.getpass('Password: ')

#         con = psycopg2.connect(dbname=conn_info["dbname"],
#                                host=conn_info["sqlhost"],
#                                port=conn_info["sqlport"],
#                                user=conn_info["sqluser"],
#                                password=conn_info["sqlpass"])
# query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres


In [3]:
# from sqlalchemy import create_engine
# con= create_engine('postgresql://eicu@localhost:5432/eicu')

In [5]:
# query = query_schema + """
# select  *
# from patient
# """
# df_total = pd.read_sql_query(query, con)

# n_patient_icu = df_total['patientunitstayid'].nunique()
# print("There are {} patients in the icu record.".format(n_patient_icu))


There are 200859 patients in the icu record.


### count the number of times that each patient into the ICU 

In [6]:
# query = query_schema + """
# with frequency_count as (
# select  uniquepid as uniquep, count(*) as frequency
# from patient
# group by uniquep
# )
# select 
#     CASE 
#         when frequency > 4 then 'Greater than 4'
#         else cast(frequency as TEXT)
#     END as frequency_category,  count(*) as occurrence
# from     frequency_count
# group by frequency_category
# order by occurrence DESC
# """
# df_unique = pd.read_sql_query(query, con)
# # df_unique['patientunitstayid'].nunique()

# df_unique['data occurrence'] = df_unique['occurrence'] / df_total['uniquepid'].nunique() * 100.0
# df_unique.sort_values('data occurrence', ascending=False, inplace=True)
# df_unique.head()

,frequency_category,occurrence,data occurrence
0,1,100884,72.387294
1,2,26554,19.053291
2,3,6612,4.744308
3,4,2899,2.080119
4,Greater than 4,2418,1.734987


### select patients that the unitadmitsource comes from  emergency and direct department

In [8]:
# query = query_schema + """

# SELECT DISTINCT patientunitstayid,hospitaladmitOffset,gender,age,apacheadmissiondx,unitadmitsource,admissionweight,	dischargeweight,unitdischargelocation,	unitdischargestatus
# FROM patient
# WHERE unitadmitsource IN ('Emergency Department', 'Direct Admit') 
#  """

# df_selected_patients = pd.read_sql_query(query, con)
# # df_selected_patients.to_csv('/Users/meisun/Documents/study/thesis/master project/data-preprocessing/selected_patients.csv', index=False)
# # print("Data exported successfully to 'selected_patients.csv'.")
# df_selected_patients.head()

,patientunitstayid,hospitaladmitoffset,gender,age,apacheadmissiondx,unitadmitsource,admissionweight,dischargeweight,unitdischargelocation,unitdischargestatus
0,141168,0,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired
1,141178,-14,Female,52,,Emergency Department,54.4,54.4,Step-Down Unit (SDU),Alive
2,141197,-25,Male,71,"Sepsis, pulmonary",Emergency Department,102.1,102.1,Step-Down Unit (SDU),Alive
3,141208,-1,Female,25,"Overdose, sedatives, hypnotics, antipsychotics...",Emergency Department,95.3,95.3,Home,Alive
4,141229,-4,Female,> 89,"CHF, congestive heart failure",Emergency Department,89.8,86.6,Floor,Alive


In [9]:
# n_selected_patients=df_selected_patients['patientunitstayid'].nunique()
# print("There are {} unique patientunitstayid in the selected patients with the selected unitadmitsource: emergency and direct.".format(n_selected_patients))

There are 102266 unique patientunitstayid in the selected patients with the selected unitadmitsource: emergency and direct.
